In [1]:
import os
import pandas as pd
import logging
import asyncio
from dotenv import load_dotenv
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, CallbackContext
from openai import OpenAI

# Load environment variables
load_dotenv()
TELEGRAM_BOT_TOKEN = os.getenv("TELEGRAM_BOT_TOKEN")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Initialize OpenAI client
client = OpenAI(api_key=OPENAI_API_KEY)

# Load machine issue data
machine_df = pd.read_csv("machine_issues.csv")

# Set up logging
logging.basicConfig(format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", level=logging.INFO)

def ask_chatgpt(prompt):
    """Send a query to OpenAI's ChatGPT and return the response."""
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an expert quality control engineer."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

def get_qc_summary(machine_id):
    """Fetch machine QC issues and generate a ChatGPT-powered analysis."""
    issues = machine_df[machine_df["machine_id"] == machine_id]

    if issues.empty:
        return f"❌ **Machine ID {machine_id} not found in records.** Please check the ID and try again."

    summary = f"🏭 **Machine QC Summary: {machine_id}**\n\n"

    for idx, row in issues.iterrows():
        issue_name = row["issue"]
        category = row["category"]
        impact = row["impact"]

        # Generate AI-driven root cause and resolution
        chatgpt_prompt = f"""
        As an experienced quality control engineer, analyze the following issue and suggest possible root causes,
        resolutions, and preventive measures.

        **Issue:** {issue_name}
        **Category:** {category}
        **Impact:** {impact}

        Provide a detailed and structured response.
        """
        ai_response = ask_chatgpt(chatgpt_prompt)

        summary += f"🔴 **Issue {idx+1}: {issue_name}**\n"
        summary += f"- **Category:** {category}\n"
        summary += f"- **Impact:** {impact}\n\n"
        summary += f"💡 **AI-Generated Analysis:**\n{ai_response}\n\n"
        summary += "---\n"

    summary += "\n📌 **Ensure preventive actions are taken to reduce future occurrences.**\n"
    return summary

async def start(update: Update, context: CallbackContext) -> None:
    """Send a welcome message when the user starts the bot."""
    await update.message.reply_text("🤖 Hello! I am the **QC AI Chatbot**.\nSend me a machine ID, and I will provide a detailed issue report using AI.")

async def handle_message(update: Update, context: CallbackContext) -> None:
    """Handle incoming messages and provide machine issue reports."""
    machine_id = update.message.text.strip().upper()
    response = get_qc_summary(machine_id)
    await update.message.reply_text(response, parse_mode="Markdown")

async def main():
    """Start the Telegram bot (Jupyter Notebook compatible)."""
    app = Application.builder().token(TELEGRAM_BOT_TOKEN).build()
    app.add_handler(CommandHandler("start", start))
    app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))
    
    print("🚀 Bot is running...")
    await app.run_polling()

# Run bot asynchronously in Jupyter Notebook
asyncio.create_task(main())


<Task pending name='Task-5' coro=<main() running at C:\Users\Deepak Reji\AppData\Local\Temp\ipykernel_1336\689137582.py:81>>

🚀 Bot is running...


In [2]:
Start

NameError: name 'Start' is not defined